# Example of Kmeans for SAMPLE_SIZE molecules and molecules with size MOL_SIZE_OF_INTEREST

In [47]:
import os.path
from decimal import Decimal
from rdkit import Chem
from rdkit import RDLogger
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn
import pickle
import os
import sklearn
RDLogger.DisableLog('rdApp.*')
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin

## SET CONSTANTS

In [48]:
DATA_FOLDER = '/home/artem/dataset_gdb17'
SAMPLE_SIZE = 100000  #<-- data set size
MOL_SIZE_OF_INTEREST = 17  # number of atoms
#DATA_FILE_NAME = 'GDB17.50000000.smi'
DATA_FILE_NAME = str(MOL_SIZE_OF_INTEREST) + '.smi'
DATA_PATH =  DATA_FOLDER + '/' + str(SAMPLE_SIZE) + '/' + DATA_FILE_NAME

_N_CLUSTERS = 100  # we want 100 mols. Not more, not fewer --> 'protected'
METRICS = 'euclidean'


In [49]:
#estimates exe time
def timeit(fn):
    """
    measures execution time. used as decorator
    @rtype: object
    """
    from time import perf_counter
    from functools import wraps

    @wraps(fn)
    def inner(*args, **kwargs):
        start = perf_counter()
        result = fn(*args, **kwargs)
        end = perf_counter()
        elapsed = end - start

        args_ = [str(a) for a in args]
        kwargs_ = ['{0}={1}'.format(k, v) for (k, v) in kwargs.items()]
        all_args = args_ + kwargs_
        args_str = ','.join(all_args)
        # print('{0}({1}) took {2:.6f}s to run.'.format(fn.__name__,
        #                                               args_str,
        #                                               elapsed))
        print('{0} took {1:.6f}s to run.'.format(fn.__name__, elapsed))
        return result

    return inner

## DATA

In [50]:
DATA_PATH

'/home/artem/dataset_gdb17/100000/17.smi'

## LOAD DATA


In [51]:
suppl = Chem.SmilesMolSupplier(DATA_PATH)
smiles_list = [Chem.MolToSmiles(m) for m in suppl]
smiles_list[0:3]
# smiles_list is our raw data. now we need to convert them into some representation

['C#Cc1c(OC=NC)[nH]c(=N)c(NCC)c1N',
 'C#CC1C(N)CC2OC(C)C(C)(N)C2C1CO',
 'CC1N=C2N3C4C5C4C(CC5C13C#N)S2(=O)=O']

## GET FEATURES FOR EVERY MOLECULE


In [52]:
features = [Chem.RDKFingerprint(m) for m in suppl]

In [53]:
fp_array = np.array([[int(i) for i in Chem.RDKFingerprint(m).ToBitString()] for m in suppl], dtype=int)

In [54]:
fp_array

array([[1, 0, 1, ..., 1, 1, 1],
       [1, 0, 0, ..., 0, 1, 1],
       [1, 0, 1, ..., 0, 0, 1],
       ...,
       [1, 0, 1, ..., 0, 0, 1],
       [1, 0, 0, ..., 0, 0, 1],
       [0, 1, 1, ..., 0, 1, 1]])

## KMEANS

In [55]:
@timeit
def returns_kmeans(n_clusters, fp_array):
    return KMeans(n_clusters=_N_CLUSTERS).fit(X=fp_array)

kmeans = returns_kmeans(n_clusters=_N_CLUSTERS, fp_array=fp_array)

returns_kmeans took 594.096937s to run.


In [56]:
kmeans.labels_

kmeans.cluster_centers_

array([[0.24373119, 0.31594784, 0.62688064, ..., 0.22567703, 0.34804413,
        0.92577733],
       [0.60503145, 0.08301887, 0.68050314, ..., 0.06792453, 0.13836478,
        1.        ],
       [0.26278837, 0.67101304, 0.8114343 , ..., 0.00601805, 0.83650953,
        1.        ],
       ...,
       [0.30267857, 0.1125    , 0.25535714, ..., 0.04553571, 0.14732143,
        1.        ],
       [0.65355191, 0.42622951, 0.4557377 , ..., 0.63606557, 0.60655738,
        0.99672131],
       [0.19298246, 0.36682616, 0.65709729, ..., 0.22328549, 0.24561404,
        1.        ]])

In [57]:
central_mol_ids = pairwise_distances_argmin(X=kmeans.cluster_centers_, Y=fp_array, metric=METRICS)

## SAVE

In [58]:
# - as a list of mol idx
fname_wo_extension =f'{DATA_FOLDER}/{str(SAMPLE_SIZE)}/{MOL_SIZE_OF_INTEREST}_{METRICS}'
np.savetxt(f'{fname_wo_extension}.txt', central_mol_ids, fmt='%i')
# - as smiles
list_of_central_smiles = [Chem.MolToSmiles(suppl[int(mol_id)]) for mol_id in central_mol_ids]
print(list_of_central_smiles[0:2])
list_of_central_smiles

current_path = f'{fname_wo_extension}.smi'
with Chem.SmilesWriter(current_path) as w:
    for mol_id in central_mol_ids:
        w.write(suppl[int(mol_id)])
print('I am done')

['CCCOCCC(C)CCCOCC(O)CN', 'CC(N)CCC(C)C1C(C)N=C(N)C(C)C1O']
I am done
